# Import

In [1]:
%pip install -U pip
%pip install llmcompressor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 69.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 83.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 31.3 MB/s  0:00:00
  Attempting uninstall: nvidia-ml-py
    Found existing installation: nvidia-ml-py 13.590.48
    Uninstalling nvidia-ml-py-13.590.48:
      Successfully uninstalled nvidia-ml-py-13.590.480/7 [nvidia-ml-py]
  Attempting uninstall: tqdm━━━━━━━━━━━━━━━━━━━━ 0/7 [nvidia-ml-py]
    Found existing installation: tqdm 4.67.2 0/7 [nvidia-ml-py]
    Uninstalling tqdm-4.67.2:━━━━━━━━━━━━━━━ 0/7 [nvidia-ml-py]
      Successfully uninstalled tqdm-4.67.2━━ 0/7 [nvidia-ml-py]
  Attempting uninstall: huggingface_hub━━━━━ 0/7 [nvidia-ml-py]
    Found existing installation: huggingface_hub 1.3.7━━━━━━━━━━━━ 3/7 [huggingface_hub]
    Uni

In [3]:
import os
import torch
import shutil
from pathlib import Path

from google.colab import drive
drive.mount('/content/drive')

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier

Mounted at /content/drive


# Setting

In [5]:
MODEL_ID = "/content/drive/MyDrive/LGAimers/base_model"
OUT_DIR  = "/content/drive/MyDrive/LGAimers/model"

DATASET_ID = "LGAI-EXAONE/MANTA-1M"
DATASET_SPLIT = "train"

NUM_CALIBRATION_SAMPLES = 256
MAX_SEQUENCE_LENGTH = 512

# Quantization
SCHEME = "W4A16"
TARGETS = ["Linear"]
IGNORE  = ["embed_tokens", "lm_head"]

# Model Loads

In [6]:
print("[INFO] 모델 로드 중...")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
)

print("[INFO] 모델/토크나이저 로드 완료")

[INFO] 모델 로드 중...


`torch_dtype` is deprecated! Use `dtype` instead!


[INFO] 모델/토크나이저 로드 완료


# Dataset Loads & Preprocess

In [7]:
print("[INFO] 캘리브레이션 데이터 로드 중...")

ds = load_dataset(
    DATASET_ID,
    split=f"{DATASET_SPLIT}[:{NUM_CALIBRATION_SAMPLES}]",
)

def preprocess(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["conversations"],
            add_generation_prompt=True,
            tokenize=False)
    }

ds = ds.map(preprocess)

print("[INFO] 데이터 전처리 완료")

[INFO] 캘리브레이션 데이터 로드 중...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train.parquet:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

[INFO] 데이터 전처리 완료


# GPTQ Quantization

In [ ]:
print(f"[INFO] GPTQ 시작 (scheme={SCHEME}, samples={NUM_CALIBRATION_SAMPLES}, max_len={MAX_SEQUENCE_LENGTH})...")

recipe = [
    GPTQModifier(
        scheme=SCHEME,
        targets=TARGETS,
        ignore=IGNORE,
    )
]

oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

print("[INFO] GPTQ 완료")

# Model Save

In [ ]:
os.makedirs(OUT_DIR, exist_ok=True)

model.save_pretrained(OUT_DIR, save_compressed=True)
tokenizer.save_pretrained(OUT_DIR)

print(f"[INFO] 모델 저장 완료: {OUT_DIR}")

# Submission

In [ ]:
zip_name = "baseline_submit"
print(f"[INFO] {zip_name}.zip 생성 중...")

shutil.make_archive(
    base_name=zip_name,
    format="zip",
    root_dir=".",
    base_dir=OUT_DIR,
)

print(f"[INFO] 생성 완료: {zip_name}.zip")